In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import array_contains, col, explode

spark = (SparkSession.builder
         .appName("filter-data")
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/04 16:28:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df = (spark.read.format("csv")
      .option("header", "true")
      .option("nullValue", "null")
      .option("dateFormat", "LLLL d, y")
      .load("../data/netflix_titles.csv"))

In [3]:
filtered_df = df.filter(col("release_year") > 2020)
filtered_df.show()

+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s2|TV Show|       Blood & Water|                null|Ama Qamata, Khosi...|        South Africa|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|After crossing pa...|
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, Tr...|                null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, I...|To protect his fa...|
|     s4|TV Show|Jailbirds New Orl.

In [4]:
filtered_df = (
    df.filter(
        (col("country") == "United States")
        & (col("release_year") > 2020)))

filtered_df.show()

+-------+-------+--------------------+--------------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|    s10|  Movie|        The Starling|      Theodore Melfi|Melissa McCarthy,...|United States|September 24, 2021|        2021| PG-13|  104 min|    Comedies, Dramas|A woman adjusting...|
|    s16|TV Show|   Dear White People|                null|Logan Browning, B...|United States|September 22, 2021|        2021| TV-MA|4 Seasons|TV Comedies, TV D...|"Students of colo...|
|    s41|TV Show|He-Man and the Ma...|                null|Yuri Lowent

In [5]:
filtered_df = (
    df.filter(
        col("country")
        .isin(["United States", "United Kingdom",  "India"])))
filtered_df.show(3)

+-------+-------+--------------------+---------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|       director|                cast|       country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+---------------+--------------------+--------------+------------------+------------+------+---------+--------------------+--------------------+
|     s1|  Movie|Dick Johnson Is Dead|Kirsten Johnson|                null| United States|September 25, 2021|        2020| PG-13|   90 min|       Documentaries|As her father nea...|
|     s5|TV Show|        Kota Factory|           null|Mayur More, Jiten...|         India|September 24, 2021|        2021| TV-MA|2 Seasons|International TV ...|In a city of coac...|
|     s9|TV Show|The Great British...|Andy Devonshire|Mel Giedroyc, Sue...|United Kingdom|

### Filtering on string

In [6]:
# filter the DataFrame based on a substring match
filtered_df = df.filter(col("listed_in").like("%Crime%"))

# display the filtered DataFrame
filtered_df.show()

+-------+-------+--------------------+--------------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|            director|                cast|      country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+--------------------+--------------------+-------------+------------------+------------+------+---------+--------------------+--------------------+
|     s3|TV Show|           Ganglands|     Julien Leclercq|Sami Bouajila, Tr...|         null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, I...|To protect his fa...|
|    s11|TV Show|Vendetta: Truth, ...|                null|                null|         null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, D...|"Sicily boasts a ...|
|    s12|TV Show|    Bangkok Breaking|   Kongkiat Komesiri|Sukollawat 

In [7]:
# filter the DataFrame based on a regular expression match
filtered_df = df.filter(col("listed_in").rlike("(Crime|Thrillers)"))

# display the filtered DataFrame
filtered_df.show()

+-------+-------+--------------------+-----------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|show_id|   type|               title|         director|                cast|             country|        date_added|release_year|rating| duration|           listed_in|         description|
+-------+-------+--------------------+-----------------+--------------------+--------------------+------------------+------------+------+---------+--------------------+--------------------+
|     s3|TV Show|           Ganglands|  Julien Leclercq|Sami Bouajila, Tr...|                null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, I...|To protect his fa...|
|    s11|TV Show|Vendetta: Truth, ...|             null|                null|                null|September 24, 2021|        2021| TV-MA| 1 Season|Crime TV Shows, D...|"Sicily boasts a ...|
|    s12|TV Show|    Bangkok Breaking|Kongkiat Kom

### Filtering on Data Ranges

In [8]:
# filter the DataFrame based on a date range
filtered_df = df.filter((col("date_added") >= "2021-09-05") & (col("date_added") <= "2021-09-01"))

# display the filtered DataFrame
filtered_df.show()

+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



In [9]:
# filter the DataFrame based on a date range
filtered_df = df.filter((col("date_added").between("2021-02-01","2021-03-01")))

# display the filtered DataFrame
filtered_df.show()

+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
|show_id|type|title|director|cast|country|date_added|release_year|rating|duration|listed_in|description|
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+
+-------+----+-----+--------+----+-------+----------+------------+------+--------+---------+-----------+



### Filter on Arrays

In [10]:
# Read parquet file into a DataFrame
df_recipes = (spark.read
      .format("parquet")
      .load("../data/recipes.parquet"))

# filter the DataFrame based on a value in the array column
filtered_df = df_recipes.filter(array_contains(col("RecipeIngredientParts"), "apple"))

# display the filtered DataFrame
filtered_df.show()


+--------+--------------------+----------+-----------------+--------+--------+---------+--------------------+--------------+--------------------+--------------------------+---------------------+----------------+-----------+--------+----------+-------------------+------------------+-------------+-------------------+------------+------------+--------------+--------------+-----------+--------------------+--------------------+-------------+
|RecipeId|                Name|  AuthorId|       AuthorName|CookTime|PrepTime|TotalTime|         Description|RecipeCategory|            Keywords|RecipeIngredientQuantities|RecipeIngredientParts|AggregatedRating|ReviewCount|Calories|FatContent|SaturatedFatContent|CholesterolContent|SodiumContent|CarbohydrateContent|FiberContent|SugarContent|ProteinContent|RecipeServings|RecipeYield|  RecipeInstructions|              Images|DatePublished|
+--------+--------------------+----------+-----------------+--------+--------+---------+--------------------+---------

### Filtering on map columns

In [11]:
# Read JSON file into a DataFrame
df_nobel_prizes = (spark.read
      .format("json")
      .option("multiLine", "true")
      .load("../data/nobel_prizes.json"))

df_nobel_prizes_exploded = (
    df_nobel_prizes
    .withColumn("laureates",explode(col("laureates"))) # Explode the laureates array column into rows
    .select(col("category")
            , col("year")
            , col("overallMotivation")
            , col("laureates"))) # Use dot notion for columns in the STRUCT field

filtered_df = (
    df_nobel_prizes_exploded
    .filter(
        (col("laureates").getItem("firstname") == "Albert") 
        & (col("laureates").getItem("surname") == "Einstein")))

filtered_df.show(truncate=False)

+--------+----+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|category|year|overallMotivation|laureates                                                                                                                                    |
+--------+----+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------+
|physics |1921|null             |{Albert, 26, "for his services to Theoretical Physics, and especially for his discovery of the law of the photoelectric effect", 1, Einstein}|
+--------+----+-----------------+---------------------------------------------------------------------------------------------------------------------------------------------+



In [12]:
spark.stop()